## Import libraries

In [ ]:
import numpy as np # for processing of arrays
import pandas as pd
import sklearn # to display model performance on test set
import statistics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt # to display images from dataset
import os
from glob import glob
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

# import tensorflow backend and keras api
import tensorflow as tf
import keras
import keras.backend as K

# import model layers and InceptionV3 architecture
from tensorflow.python.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.ensemble import RandomForestClassifier

# import optimizers and callbacks
from keras.optimizers import SGD, Adadelta, RMSprop, Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from tensorflow.python.keras.layers import VersionAwareLayers

layers = VersionAwareLayers()
Dropout = layers.Dropout
Dense = layers.Dense
Input = layers.Input
concatenate = layers.concatenate
GlobalAveragePooling2D = layers.GlobalAveragePooling2D
AveragePooling2D = layers.AveragePooling2D
Flatten = layers.Flatten

In [ ]:
# Set seed
seed_value = 1402
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)


## Check if CUDA is available
CUDA is a parallel computing platform which can speed up our computations (i.e. train this model much faster!) and can be used on any Nvidia GPU from the G8x series onwards.

If CUDA is unavailable, check that your runtime has been changed to GPU. (Runtime -> Change runtime type -> GPU)

In [ ]:
print("GPU:", tf.config.list_physical_devices('GPU'), "\nCUDA Enabled:", tf.test.is_built_with_cuda(), "\nGPU Name:", tf.test.gpu_device_name(), "\nVisible Devices:", tf.config.experimental.list_physical_devices('GPU'))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] 
CUDA Enabled: True 
GPU Name: /device:GPU:0 
Visible Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12949610930262390658
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7250706432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1442141841973699400
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"
]


## Download dataset

In [ ]:
!wget -nv -q --show-progress -O ff.zip https://bit.ly/3w0xyBl
!unzip -q ff.zip -d ffdata
!rm ff.zip

ff.zip              100%[===================>]   1.75G  51.1MB/s    in 47s     


In [ ]:
# ImageDataGenerator loads images into memory in batches of specified size (in this case 16 images per batch)
# this avoids possible memory issues
train_folder = '/content/ffdata/train'
val_folder = '/content/ffdata/val'
df_train = pd.read_csv(train_folder + '/image_labels.csv')
df_val = pd.read_csv(val_folder + '/image_labels.csv')

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # rescaling normalizes pixel values from the range [0,255] to [0,1]
train_set = datagen.flow_from_dataframe(dataframe=df_train, directory=train_folder, classes= ['real', 'fake'], class_mode="categorical", target_size=(299, 299), batch_size=16)
val_set = datagen.flow_from_dataframe(dataframe=df_val, directory=val_folder, classes= ['real', 'fake'], class_mode="categorical", target_size=(299, 299), batch_size=16)

print("Check class name mapping to label index:")
print(train_set.class_indices)
print(val_set.class_indices)

Found 66722 validated image filenames belonging to 2 classes.
Found 12592 validated image filenames belonging to 2 classes.
Check class name mapping to label index:
{'fake': 0, 'real': 1}
{'fake': 0, 'real': 1}


## Model architecture

InceptionV3 is a convolutional neural network, meaning that it extracts features (in this case, faces and facial features) from input images using matrix operations (convolutions)

### Common Layers used in InceptionV3

### Load InceptionV3 Pretrained Model
> InceptionV3 documentation: https://keras.io/api/applications/inceptionv3/

In [ ]:
# base_model = InceptionV3(include_top=False, weights="imagenet", input_tensor=Input(shape=(299,299,3)))
# # add global pooling and dense layers to obtain output from the model
# layer = base_model
# layer = GlobalAveragePooling2D()(layer.output)
# layer = Dense(2, activation='softmax', name='output')(layer)
# input_layer = base_model.input
# model = Model(inputs=input_layer, outputs=layer, name="InceptionV3")


base_model = tf.keras.applications.Xception(input_shape=(299, 299, 3),
                                            include_top=False,
                                            weights='imagenet')

layer = base_model
layer = GlobalAveragePooling2D()(layer.output)
layer = Dense(2, activation='sigmoid', name='output')(layer)
input_layer = base_model.input
model = Model(inputs=input_layer, outputs=layer, name="Xception")


83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
# display model summary
model.summary()

Model: "Xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

<a name="Optimisers"></a>
### Optimiser
> Updates model in response to output of loss function (loss function tells the model the error in classification of individual samples)

In [ ]:
# # optimisers from Keras https://keras.io/api/optimizers/#available-optimizers
# sgd = SGD(learning_rate=0.001, momentum=0.9, nesterov=False)
# adadelta = Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07)
# rmsprop = RMSprop(learning_rate=0.001, rho=0.9, momentum=0.9, epsilon=1e-07, centered=False)

#compile model
# model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(optimizer = Adam(learning_rate=0.0001),
              loss ='binary_crossentropy',
              metrics = ['accuracy'])

# Add early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 20)


## Define callbacks

In [ ]:
!mkdir "/content/weights/"

In [ ]:
# ModelCheckpoint callback saves the model weights after every epoch (iteration through the dataset)
# if the validation accuracy is higher than that of the model previously saved
#checkpoint = ModelCheckpoint("/content/weights/inceptionv3.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

checkpoint = ModelCheckpoint("/content/weights/xception.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

## Train model
> Fit the model to the data

In [ ]:
#hist = model.fit(train_set, steps_per_epoch=16, epochs = 80, validation_data=val_set, validation_steps=8, callbacks = [checkpoint, es])

Epoch 1/80
16/16 [==============================] - 13s 826ms/step - loss: 0.2037 - accuracy: 0.9102 - val_loss: 0.4756 - val_accuracy: 0.8594

Epoch 00001: val_loss did not improve from 0.22443
Epoch 2/80
16/16 [==============================] - 13s 829ms/step - loss: 0.2632 - accuracy: 0.8945 - val_loss: 0.7214 - val_accuracy: 0.7344

Epoch 00002: val_loss did not improve from 0.22443
Epoch 3/80
16/16 [==============================] - 13s 846ms/step - loss: 0.2624 - accuracy: 0.8906 - val_loss: 0.3500 - val_accuracy: 0.8750

Epoch 00003: val_loss did not improve from 0.22443
Epoch 4/80
16/16 [==============================] - 14s 867ms/step - loss: 0.1998 - accuracy: 0.9102 - val_loss: 0.4903 - val_accuracy: 0.8359

Epoch 00004: val_loss did not improve from 0.22443
Epoch 5/80
16/16 [==============================] - 14s 857ms/step - loss: 0.3037 - accuracy: 0.8672 - val_loss: 0.5441 - val_accuracy: 0.7812

Epoch 00005: val_loss did not improve from 0.22443
Epoch 6/80
16/16 [=======

## Load Weights

In [ ]:
model.load_weights("/content/weights/xception.h5")

<a name="eval"></a>
## Evaluate Model

To classify the video as real or fake, we will first obtain the model's predictions on each frame extracted from the video. We then take the mean of the prediction (probability) of the frames as the prediction of the video. 

In [ ]:
def read_image_from_disk(path):

  """
  Helper function to read image from disk given a absolute path.

  :param path: Absolute path to image file on disk
  :return: Image in Numpy Ndarray representation
  """

  img = tf.keras.preprocessing.image.load_img(path, target_size=(299,299,3))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = img/255
  img = np.expand_dims(img, axis=0)

  return img


def get_frames_to_vid_mapping(frame_list):

  """
  Helper function to generate a mapping of frames to it's corresponding video 
  name.

  The path of frames in the frame_list will be in such format:
  image/[video name]/[frame number].jpg
  e.g. image/00000/00032.jpg

  :param frame_list: A list of paths to the image frames
  :return: A sorted dictionary with keys as the video name and value as the
           corresponding frames.
           e.g. of returned mapping dictionary:

            {
              "00000":[
                  "00032",
                  "00064",
                  .
                  .
                  .
                  "00487"
              ],
              "00001":[
                  "00000",
                  "00032",
                  .
                  .
                  .
                  "00392"
              ],
              .
              .
              .
              "00790":[
                  "00000",
                  "00027",
                  .
                  .
                  .
                  "00542"
              ]
            }
  """

  # Get all videos name
  vidnames = [frame.split("/")[1:2][0] for frame in frame_list]
  # Get only unique names
  vidnames = set(vidnames)
  # Init the mapping dict
  mapping = {vidname: [] for vidname in vidnames}

  # Add frames to to its corresponding list
  for frame in frame_list:
    vidname = str(frame.split("/")[1:2][0])
    frame_number = str(frame.split("/")[-1].split(".")[0])
    mapping[vidname].append(frame_number)

  return dict(sorted(mapping.items()))


def infer_videos(test_data_path, csv_file, num_of_videos='All'):

  """
  Function to infer a test data set. The function takes in a path to the test
  data set and a csv file that contain the paths of the frames extracted from 
  the videos in the test dataset.

  :param test_data_path: Absoulute path to the test dataset
  :param csv_file: File Name of the CSV file that must be in the test_data_path
  :param num_of_videos: Number of videos to infer from the dataset (default: All)
  :return: Pandas dataframe which contains the prediction (probability of being 
           fake) of each video. 
  """

  list_dir = list(pd.read_csv(test_data_path + csv_file).iloc[:,0])

  mapping = get_frames_to_vid_mapping(list_dir)

  # [*mapping] gives the list of keys (video name) in the mapping dict
  num_of_videos_avail = len([*mapping])

  # Set number of videos to be inferred to total of videos available if given 
  # num_of_videos is more than max amount of available videos
  if num_of_videos == 'All' or num_of_videos > num_of_videos_avail:
      num_of_videos = num_of_videos_avail

  # init mapping of videos to its corresponding predicted probabilities
  videos_to_prediction = {}

  # Loop through each video and make a prediction of each frame in the video.
  # Assigned a prediction to each video by taking the mean of its corresponding
  # frames' probabilities.
  for video_name in [*mapping][0:num_of_videos]:

    frames = mapping[video_name]
    predictions = []
    print("Infering video {video}...".format(video=video_name))
    print("Processing frame ", end=" ")

    # Process each frame in video
    for frame in frames:
      print(frame, end =", ")
      frame_path = "image/{video_name}/{frame}.jpg".format(video_name=video_name, frame=frame)
      img = read_image_from_disk(test_data_path + frame_path)
      prediction = model.predict(img)[0]
      # Collect only the 'real' side of probability
      predictions.append(prediction[1])

    # Take the mean of the probabilities from the frames
    videos_to_prediction[video_name] = statistics.mean(predictions)
    print("Done!")
  
  return pd.DataFrame(videos_to_prediction.items())


In [ ]:
modelPredictions = infer_videos("/content/ffdata/test/", "image_labels.csv")


Infering video 00000...
Processing frame  00032, 00064, 00097, 00129, 00162, 00194, 00227, 00259, 00292, 00324, 00357, 00389, 00422, 00454, 00487, Done!
Infering video 00001...
Processing frame  00000, 00032, 00065, 00098, 00130, 00163, 00196, 00228, 00261, 00294, 00326, 00359, 00392, Done!
Infering video 00002...
Processing frame  00000, 00027, 00054, 00081, 00108, 00135, 00162, 00189, 00216, 00243, 00271, 00298, 00325, 00352, 00379, 00406, 00433, 00460, 00487, 00514, 00542, Done!
Infering video 00003...
Processing frame  00000, 00031, 00063, 00095, 00127, 00159, 00191, 00222, 00254, 00286, 00318, 00350, 00382, 00413, 00445, 00477, 00509, 00541, 00573, 00605, Done!
Infering video 00004...
Processing frame  00000, 00026, 00052, 00078, 00105, 00131, 00157, 00183, 00210, 00236, 00262, 00289, 00315, 00341, 00367, 00394, 00420, 00446, 00472, 00499, 00525, 00551, 00578, 00604, 00630, 00656, 00683, 00709, 00735, 00762, Done!
Infering video 00005...
Processing frame  00000, 00025, 00051, 0007

In [ ]:
modelPredictions

,0,1
0,00000,0.000346
1,00001,0.118979
2,00002,0.110075
3,00003,0.236406
4,00004,0.078750
...,...,...
785,00785,0.017115
786,00786,0.205811
787,00787,0.082880
788,00788,0.037370


## Submission

In [ ]:
modelPredictions.columns = ['vid_name', 'label']
modelPredictions.to_csv("/content/ffdata/Lim Xi Chen Terry_terryworld1998@gmail.com.csv", index=False)